#### BOW,TFIDF,Machine Learning Algorithms
(order very imp***)
1. Preprocessing And Cleaning
2. Train Test Split
3. BOW And TF-IDF (Sentences--->vectors) {Preventing Data Leakage}
4. Trained Our Models


In [1]:
import pandas as pd
messages=pd.read_csv('../smsspamcollection/SMSSpamCollection', sep='\t',names=["label","message"])

In [2]:
messages

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [3]:
## Data Cleaning And Preprocessing
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\salma\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
wnl=WordNetLemmatizer()

In [5]:
corpus=[]
for i in range(0,len(messages)):
    review=re.sub('[^a-zA-z]',' ',messages['message'][i])
    review=review.lower()
    review=review.split()
    review=[wnl.lemmatize(word, pos='v') for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

In [6]:
corpus[0:4]

['go jurong point crazy available bugis n great world la e buffet cine get amore wat',
 'ok lar joke wif u oni',
 'free entry wkly comp win fa cup final tkts st may text fa receive entry question std txt rate c apply',
 'u dun say early hor u c already say']

In [56]:
## Output Features
y=pd.get_dummies(messages['label'])
print(y.astype(int))
y=y.iloc[:,1].values.astype(int)
y

      ham  spam
0       1     0
1       1     0
2       0     1
3       1     0
4       1     0
...   ...   ...
5567    0     1
5568    1     0
5569    1     0
5570    1     0
5571    1     0

[5572 rows x 2 columns]


array([0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0])

## method 1:

In [106]:
## Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(corpus,y,test_size=0.20)
len(X_train),len(y_train)

(4457, 4457)

In [107]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    AdaBoostClassifier,
    BaggingClassifier,
    ExtraTreesClassifier,
    GradientBoostingClassifier,
    RandomForestClassifier
)

from xgboost import XGBClassifier
classifiers = {
    "Naive Bayes": MultinomialNB(),
    "Logistic Regression": LogisticRegression(class_weight='balanced', max_iter=1000),
    "Random Forest": RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=42),
    "SVM": LinearSVC(class_weight='balanced'),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "Decision Tree": DecisionTreeClassifier(class_weight='balanced', random_state=42),
    "Extra Trees": ExtraTreesClassifier(n_estimators=100, class_weight='balanced', random_state=42),
    "AdaBoost": AdaBoostClassifier(n_estimators=100, random_state=42),
    "Bagging": BaggingClassifier(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
}

In [108]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report

scores = []
best_model = None
best_score = 0

for name, clf in classifiers.items():
    print(f"\n🧠 {name}")
    pipe = Pipeline([
        ('cv', CountVectorizer(max_features=3000, stop_words='english', ngram_range=(1, 2))),
        ('clf', clf)
    ])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    if acc > best_score:
        best_score = acc
        best_model = (name, pipe)

print("\n🏆 Best Model:")
print(f"{best_model[0]} with accuracy: {best_score:.4f}")


🧠 Naive Bayes
Accuracy: 0.9865470852017937
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       968
           1       0.95      0.95      0.95       147

    accuracy                           0.99      1115
   macro avg       0.97      0.97      0.97      1115
weighted avg       0.99      0.99      0.99      1115


🧠 Logistic Regression
Accuracy: 0.9865470852017937
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       968
           1       0.95      0.95      0.95       147

    accuracy                           0.99      1115
   macro avg       0.97      0.97      0.97      1115
weighted avg       0.99      0.99      0.99      1115


🧠 Random Forest
Accuracy: 0.9820627802690582
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       968
           1       0.98      0.88      0.93       147

    accuracy                           0

c:\Users\salma\PycharmProjects\Krish-Naik-course-resources\Complete NLP Bootcamp\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [23:20:04] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Accuracy: 0.97847533632287
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       968
           1       0.94      0.89      0.92       147

    accuracy                           0.98      1115
   macro avg       0.96      0.94      0.95      1115
weighted avg       0.98      0.98      0.98      1115


🏆 Best Model:
Naive Bayes with accuracy: 0.9865


In [109]:
best_model

('Naive Bayes',
 Pipeline(steps=[('cv',
                  CountVectorizer(max_features=3000, ngram_range=(1, 2),
                                  stop_words='english')),
                 ('clf', MultinomialNB())]))

In [110]:
best_model[1].fit(X_train, y_train)
y_pred = best_model[1].predict(X_test)

In [111]:
test_text = "Congratulations! You've been selected to receive a free iPhone 🎉 Click here to claim your prize now!"
test_text=re.sub('[^a-zA-z]',' ',test_text).lower().split()
test_text=[wnl.lemmatize(word, pos='v') for word in test_text if not word in stopwords.words('english')]
test_text=[' '.join(test_text)]
result = best_model[1].predict(test_text)[0]
print(result)
print("Spam" if result == 1 else "Ham")

1
Spam


In [112]:
test_text = "Hey, are we still meeting at 5 PM today? Let me know if anything changes."
test_text=re.sub('[^a-zA-z]',' ',test_text).lower().split()
test_text=[wnl.lemmatize(word, pos='v') for word in test_text if not word in stopwords.words('english')]
test_text=[' '.join(test_text)]
result = best_model[1].predict(test_text)[0]
print(result)
print("Spam" if result == 1 else "Ham")

0
Ham


## method 2:
## Create Bag Of Words

In [63]:
## Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(corpus,y,test_size=0.20)
len(X_train),len(y_train)

(4457, 4457)

In [64]:
## Create the Bag OF Words model
from sklearn.feature_extraction.text import CountVectorizer
## for Binary BOW enable binary=True
cv=CountVectorizer(max_features=3000,ngram_range=(1,2))
X_train=cv.fit_transform(X_train).toarray()
X_test=cv.transform(X_test).toarray()

In [65]:
cv.vocabulary_

{'dnt': 640,
 'live': 1409,
 'lt': 1479,
 'gt': 1026,
 'cm': 388,
 'thnk': 2545,
 'different': 622,
 'lt gt': 1481,
 'boo': 199,
 'babe': 134,
 'seem': 2204,
 'gettin': 949,
 'well': 2866,
 'hunny': 1182,
 'hope': 1148,
 'ure': 2746,
 'ok': 1773,
 'take': 2465,
 'care': 310,
 'xxxx': 2957,
 'ok take': 1781,
 'take care': 2466,
 'date': 565,
 'service': 2231,
 'ask': 110,
 'contact': 460,
 'someone': 2326,
 'call': 247,
 'reveal': 2119,
 'pobox': 1944,
 'uz': 2772,
 'date service': 566,
 'call reveal': 274,
 'pobox uz': 1946,
 'lol': 1427,
 'record': 2064,
 'one': 1790,
 'night': 1724,
 'yar': 2963,
 'want': 2820,
 'yest': 2982,
 'late': 1326,
 'already': 67,
 'get': 913,
 'go': 972,
 'lor': 1440,
 'still': 2395,
 'act': 20,
 'real': 2050,
 'ask go': 111,
 'go lor': 980,
 'actually': 23,
 'quite': 2019,
 'fast': 757,
 'cos': 487,
 'da': 553,
 'ge': 903,
 'slow': 2297,
 'wat': 2832,
 'haha': 1054,
 'yar lor': 2964,
 'da ge': 554,
 'channel': 346,
 'might': 1570,
 'earlier': 678,
 'reach'

In [66]:
import numpy as np
np.set_printoptions(edgeitems=30, linewidth=100000, formatter=dict(float=lambda x: "%.3g" % x))
X_train

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0

In [67]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model=MultinomialNB().fit(X_train,y_train)
y_pred=spam_detect_model.predict(X_test)

In [68]:
from sklearn.metrics import accuracy_score, classification_report
accuracy_score(y_test,y_pred)

0.9874439461883409

In [69]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       962
           1       0.98      0.93      0.95       153

    accuracy                           0.99      1115
   macro avg       0.98      0.96      0.97      1115
weighted avg       0.99      0.99      0.99      1115



In [86]:
test_text = "Congratulations! You've been selected to receive a free iPhone 🎉 Click here to claim your prize now!"
test_text=re.sub('[^a-zA-z]',' ',test_text).lower().split()
test_text=[wnl.lemmatize(word, pos='v') for word in test_text if not word in stopwords.words('english')]
test_text=[' '.join(test_text)]
test_text = cv.transform(test_text).toarray()
result = spam_detect_model.predict(test_text)[0]
print(result)
print("Spam" if result == 1 else "Ham")

1
Spam


In [87]:
test_text = "Hey, are we still meeting at 5 PM today? Let me know if anything changes."
test_text=re.sub('[^a-zA-z]',' ',test_text).lower().split()
test_text=[wnl.lemmatize(word, pos='v') for word in test_text if not word in stopwords.words('english')]
test_text=[' '.join(test_text)]
test_text = cv.transform(test_text).toarray()
result = spam_detect_model.predict(test_text)[0]
print(result)
print("Spam" if result == 1 else "Ham")

0
Ham


## method 3:

### Creating The TF-IDF Model

In [73]:
## Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(corpus,y,test_size=0.20)

In [74]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv=TfidfVectorizer(max_features=2500,ngram_range=(1,2))
X_train=tv.fit_transform(X_train).toarray()
X_test=tv.transform(X_test).toarray()

In [75]:
X_train

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.555, 0.74, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 

In [76]:
tv.vocabulary_

{'yup': 2496,
 'ok': 1492,
 'yup ok': 2497,
 'plan': 1594,
 'miss': 1323,
 'call': 234,
 'thats': 2098,
 'rule': 1781,
 'one': 1507,
 'great': 861,
 'phone': 1574,
 'receive': 1712,
 'quality': 1678,
 'win': 2393,
 'miss call': 1326,
 'wish': 2402,
 'guy': 888,
 'semester': 1829,
 'never': 1426,
 'pick': 1582,
 'pick phone': 1583,
 'tot': 2167,
 'group': 866,
 'mate': 1271,
 'lucky': 1230,
 'havent': 915,
 'reply': 1736,
 'wat': 2332,
 'time': 2129,
 'need': 1415,
 'leave': 1139,
 'wat time': 2335,
 'life': 1159,
 'mean': 1283,
 'lot': 1205,
 'love': 1209,
 'people': 1559,
 'world': 2428,
 'friends': 744,
 'ge': 775,
 'lot love': 1206,
 'love life': 1212,
 'wait': 2307,
 'still': 1979,
 'clear': 354,
 'sure': 2034,
 'want': 2320,
 'live': 1180,
 'us': 2265,
 'different': 524,
 'style': 2011,
 'da': 472,
 'go': 816,
 'talk': 2053,
 'tv': 2199,
 'give': 811,
 'plus': 1613,
 'say': 1800,
 'greet': 864,
 'whenever': 2381,
 'speak': 1946,
 'ever': 626,
 'think': 2105,
 'good': 841,
 'perfec

In [77]:
from sklearn.naive_bayes import MultinomialNB
spam_tfidf_model = MultinomialNB().fit(X_train, y_train)

In [78]:
#prediction
y_pred=spam_tfidf_model.predict(X_test)

In [79]:
score=accuracy_score(y_test,y_pred)
print(score)

0.97847533632287


In [80]:
from sklearn.metrics import classification_report
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99       990
           1       0.84      1.00      0.91       125

    accuracy                           0.98      1115
   macro avg       0.92      0.99      0.95      1115
weighted avg       0.98      0.98      0.98      1115



In [89]:
test_text = "Congratulations! You've been selected to receive a free iPhone 🎉 Click here to claim your prize now!"
test_text=re.sub('[^a-zA-z]',' ',test_text).lower().split()
test_text=[wnl.lemmatize(word, pos='v') for word in test_text if not word in stopwords.words('english')]
test_text=[' '.join(test_text)]
test_text = tv.transform(test_text).toarray()
result = spam_tfidf_model.predict(test_text)[0]
print(result)
print("Spam" if result == 1 else "Ham")

1
Spam


In [90]:
test_text = "Hey, are we still meeting at 5 PM today? Let me know if anything changes."
test_text=re.sub('[^a-zA-z]',' ',test_text).lower().split()
test_text=[wnl.lemmatize(word, pos='v') for word in test_text if not word in stopwords.words('english')]
test_text=[' '.join(test_text)]
test_text = tv.transform(test_text).toarray()
result = spam_tfidf_model.predict(test_text)[0]
print(result)
print("Spam" if result == 1 else "Ham")

0
Ham
